In [2]:
import os
import cv2
import json
import torch
import torchvision
from torch import nn
import pandas as pd
import numpy as np
from skimage import color, io
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision.datasets import MNIST
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, ConcatDataset

In [3]:
#########################
# HUMANS IMAGES
#########################

humans_path = '/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/synthetic_images'

all_humans = []
    
for root, dirs, files in sorted(os.walk(humans_path)):
    
    for name in files:
        if not name.startswith("."):
            all_humans.append(str(root) + "/" + str(name))
        
print("loaded %d files into human database" % len(all_humans))

loaded 36001 files into human database


In [4]:
#########################
# HUMANS ANNOTATIONS
#########################

humans_annotations_path = '/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/annotations'

all_humans_annotations = []
    
for root, dirs, files in sorted(os.walk(humans_annotations_path)):
    
    for name in files:
        if not name.startswith("."):
            all_humans_annotations.append(str(root) + "/" + str(name))
        
print("loaded %d files into all_humans_annotations database" % len(all_humans_annotations))

print(all_humans_annotations[0])

loaded 6000 files into all_humans_annotations database
/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/annotations/female/subject_mesh_1968_anno.json


In [5]:
##############################################################
# DATA SORTING
##############################################################

##############################
# HUMANS
##############################

humans_plain_paths = []
humans_one_paths = []
humans_jeans_paths = []

for human in all_humans:
    if "ein" in human:
        humans_one_paths.append(human)
    elif "jeans" in human:
        humans_jeans_paths.append(human)
    else:
        humans_plain_paths.append(human)
        
##############################
# ANNOTATIONS
##############################

humans_annotations_df = pd.DataFrame() 

for humans_annotation_path in all_humans_annotations:

    if not "DS_Store" in humans_annotation_path:
    
        with open(humans_annotation_path, 'r') as file:
            # load from json
            human_annotation = json.load(file)
            sub_df = pd.DataFrame.from_dict(human_annotation['human_dimensions'], orient='index')
            sub_df = sub_df.transpose()

            #humans_annotations_df = humans_annotations_df.append(sub_df, ignore_index=True)
            humans_annotations_df = pd.concat([humans_annotations_df,sub_df])
    
##############################
# OUTPUT
##############################
print("Images loaded:")
print("----------------------------------------------------")
print("Examples:")
print("----------------------------------------------------")
print(humans_jeans_paths[0])
print(humans_plain_paths[0])
print(humans_one_paths[0])
print("----------------------------------------------------")
print("Numbers:")
print("----------------------------------------------------")
print(len(humans_jeans_paths))
print(len(humans_plain_paths))
humans_one_paths = humans_one_paths[:-1]
print(len(humans_one_paths))

Images loaded:
----------------------------------------------------
Examples:
----------------------------------------------------
/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/synthetic_images/200x200/pose0/female/subject_mesh_1758-jeans.png
/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/synthetic_images/200x200/pose0/female/subject_mesh_0978.png
/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/synthetic_images/200x200/pose0/female/subject_mesh_0699-ein.png
----------------------------------------------------
Numbers:
----------------------------------------------------
12000
12000
12000


In [6]:
print(humans_annotations_df.columns)
print(humans_annotations_df.head())

Index(['chest_circumference', 'height', 'inseam', 'left_arm_length',
       'pelvis_circumference', 'right_arm_length', 'shoulder_width',
       'waist_circumference'],
      dtype='object')
   chest_circumference    height    inseam  left_arm_length  \
0             1.200267  1.730106  0.684563         0.577781   
0             0.973475  1.790180  0.841815         0.633797   
0             0.748137  1.467417  0.628197         0.486339   
0             0.882200  1.579298  0.649627         0.525917   
0             1.016914  1.456891  0.612365         0.503487   

   pelvis_circumference  right_arm_length  shoulder_width  waist_circumference  
0              1.294256          0.597256        0.382759             1.103298  
0              1.109804          0.635824        0.352502             0.814699  
0              0.902817          0.473478        0.315147             0.609579  
0              1.041371          0.528704        0.332407             0.711979  
0              0.921696  

In [7]:
##############################################################
# CONVERT IMAGES TO GRAYSCALE
##############################################################

# reduce number of images for testing purposes
n_images = 1000000

humans_plain = []
humans_jeans_grey = []
humans_one_grey = []

##############################
# HUMANS PLAIN
##############################

i = 0

for file_path in humans_plain_paths:
    human_plain = io.imread(file_path)
    
    humans_plain.append(human_plain)
       
    if i == n_images:
        break
        
    i += 1
    
print("1")
##############################
# HUMANS W TEXTURE FOREGROUND
##############################

i = 0

for file_path in humans_jeans_paths:
    human_texture_grey = color.rgb2gray(io.imread(file_path))

    humans_jeans_grey.append(human_texture_grey)

    if i == n_images:
        break

    i += 1

print("2")
##############################
# HUMANS W SOLID FOREGROUND
##############################

i = 0

for file_path in humans_one_paths:
    human_rgb_grey = color.rgb2gray(io.imread(file_path))
    
   
    humans_one_grey.append(human_rgb_grey)
       
    if i == n_images:
        break
        
    i += 1

print("3")

1
2
3


In [8]:
##############################################################
# CREATE DATA FRAME OF IMAGE VECTORS AND ANNOTATIONS
##############################################################

##############################
# ANNOTATIONS
##############################

# tensor of annotations
humans_annotation_tensor = torch.tensor(humans_annotations_df.values)

# copy annotations - pose0 and pose1 have the same annotations
humans_annotation_tensor = torch.cat((humans_annotation_tensor,humans_annotation_tensor),0)


##############################
# HUMAN PLAIN DATA FRAME
##############################

# tensor of images w.o. annotations
humans_plain_image_tensor = torch.tensor(humans_plain)

##############################
# HUMAN RGB DATA FRAME
##############################


# tensor of images w.o. annotations
humans_one_image_tensor = torch.tensor(humans_one_grey)

##############################
# HUMAN TEXTURE DATA FRAME
##############################


# tensor of images w.o. annotations
humans_jeans_image_tensor = torch.tensor(humans_jeans_grey)


############################################################
# STORE TENSORS IN FILE
############################################################

##############################
# PLAIN
##############################

torch.save(humans_annotation_tensor, '/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/tensors/humans_annotation_tensor.pt')
print("humans_annotation_tensor.pt saved")
torch.save(humans_plain_image_tensor, '/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/tensors/humans_plain_image_tensor.pt')
print("humans_plain_image_tensor.pt saved")

##############################
# ONE COLOR
##############################

torch.save(humans_one_image_tensor, '/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/tensors/humans_one_image_tensor.pt')
print("humans_one_image_tensor.pt saved")

##############################
# JEANS
##############################

torch.save(humans_jeans_image_tensor, '/Users/daniilbarkov/MyProjects/Uni/Pattern2/dataset/tensors/humans_jeans_image_tensor.pt')
print("humans_jeans_image_tensor.pt saved")

/var/folders/rl/xp7lnvk10sx90l166vcpd6p80000gn/T/ipykernel_14381/1774606740.py:21: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:233.)
  humans_plain_image_tensor = torch.tensor(humans_plain)


humans_annotation_tensor.pt saved
humans_plain_image_tensor.pt saved
humans_one_image_tensor.pt saved
humans_jeans_image_tensor.pt saved
